# Example Conversation with Tool Agent

In this example, we will show you 
- How to use the built-in tool agent and service functions
- How to use the `ServiceFactory` module pre-process the tool functions for LLMs
- How to create a new service function for the tool agent to use 

## Prerequisites

- Follow the [README.md](https://github.com/modelscope/agentscope) to install agentscope.
- Prepare a model configuration. AgentScope supports both local deployed model services (CPU or GPU) and third-party services. More details and example model configurations please refer to our [tutorial](https://modelscope.github.io/agentscope/en/tutorial/203-model.html). 
- Prepare a bing search API key.

In [9]:
BING_API_KEY = "{YOUR_BING_API_KEY}"

import agentscope

agentscope.init(model_configs="{YOUR_MODEL_CONFIG}")
YOUR_MODEL_CONFIG_NAME = "{YOUR_MODEL_CONFIG_NAME}"

## Guidance

### Step 1: Customize a new service function 

Taking `execute_python_code` as an example, we will show how to create a new service function that can be processed by `ServiceFactory` module. 

In AgentScope, a service function should have the following structure:

- A well formatted docstring (Google style is recommended), which contains
    - A brief description of function in docstring. 
    - A brief description of the input arguments. 
- Wrap the output and execution status into a `ServiceResponse` object. 

The following is a simple example of a service function that executes Python code and captures the output. (Also, you can use the `execute_python_code` function in the `agentscope.service` module directly, which provide more features.)

In [1]:
import sys
import io
from agentscope.service import ServiceResponse, ServiceExecStatus

def execute_python_code(code: str):
    """Execute Python code and capture the output. Note you must print output!
    
    Args:
        code (`str`):
            The Python code to be executed.
    """

    # Create a StringIO object to capture the output
    old_stdout = sys.stdout
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    
    try:
        # Using `exec` to execute code 
        exec(code)
    except Exception as e:
        # If an exception occurs, capture the exception information
        output = str(e)
        status = ServiceExecStatus.ERROR
    else:
        # If the execution is successful, capture the output
        output = new_stdout.getvalue()
        status = ServiceExecStatus.SUCCESS
    finally:
        # Recover the standard output
        sys.stdout = old_stdout
    
    # Wrap the output and status into a ServiceResponse object
    return ServiceResponse(status, output)

After defining the service function, we try to use `ServiceFactory` to pre-process the tool functions for LLMs.

In [2]:
from agentscope.service import ServiceFactory

func_for_agent, func_json = ServiceFactory.get(execute_python_code)

import json
print(json.dumps(func_json, indent=4))

{
    "type": "function",
    "function": {
        "name": "execute_python_code",
        "description": "Execute Python code and capture the output. Note you must `print` the output to get the result.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "The Python code to be executed."
                }
            },
            "required": [
                "code"
            ]
        }
    }
}


### Step 2: Prepare all tool functions for the agent

Similar as above, we can create different service functions, or use the built-in functions in `agentscope.service` module as follows. More service functions can be found under the `agentscope.service` module.

Note to replace the bing search API key with your own key.

In [3]:
from agentscope.service import (
    bing_search, # or google_search,
    read_text_file,
    write_text_file, 
    ServiceFactory
)

# Deal with arguments that need to be input by developers
tools = [
    ServiceFactory.get(bing_search, api_key=BING_API_KEY, num_results=3),
    ServiceFactory.get(execute_python_code),
    ServiceFactory.get(read_text_file),
    ServiceFactory.get(write_text_file),
]

Let's take a look at the tool functions we have pre-processed.

In [4]:
import json

for _, description in tools:
    print(json.dumps(description, indent=4))

{
    "type": "function",
    "function": {
        "name": "bing_search",
        "description": "Search question in Bing Search API and return the searching results",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The search query string."
                }
            },
            "required": [
                "question"
            ]
        }
    }
}
{
    "type": "function",
    "function": {
        "name": "execute_python_code",
        "description": "Execute Python code and capture the output. Note you must `print` the output to get the result.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "The Python code to be executed."
                }
            },
            "required": [
                "code"

### Step 3: Create a Tool Agent and Build a Conversation

A tool agent is built in AgentScope as an example to show how to use the service functions. However, a single tool agent and its prompt engineering strategy may not be able to handle all tasks. Developers are encouraged to create their own agents and prompt engineering strategies based on their own needs.

Here we just initialize a tool agent and print its system prompt as an example. 

Note to replace the model configuration name with yours. 

In [5]:
from agentscope.agents import ToolAgent

agent = ToolAgent(
    name="Assistant",
    model_config_name=YOUR_MODEL_CONFIG_NAME,
    tools=tools,
    verbose=True, # whether to print the raw response from LLM, execution status, and output
)

print(agent.sys_prompt)

2024-03-25 18:08:23.657 | INFO     | agentscope.models:read_model_configs:171 - Load configs for model wrapper: post_api
2024-03-25 18:08:23.660 | INFO     | agentscope.utils.monitor:_create_monitor_table:341 - Init [monitor_metrics] as the monitor table
2024-03-25 18:08:23.660 | INFO     | agentscope.utils.monitor:_create_monitor_table:342 - Init [monitor_metrics_quota_exceeded] as the monitor trigger
2024-03-25 18:08:23.660 | INFO     | agentscope.utils.monitor:__init__:311 - SqliteMonitor initialization completed at [./runs/run_20240325-180823_7jf3ed/agentscope.db]
2024-03-25 18:08:23.670 | INFO     | agentscope.models.model:__init__:256 - Initialize model [post_api]

You're a helpful assistant. You target is to help users to solve their problems.

The following tool functions are available in the format of
```
{index}. {function name}: {function description}
    {argument name} ({argument type}): {argument description}
    ...
```

Tool Functions:
1. bing_search: Search question in

Let's try some simple question here to see how the tool agent works.

The first question is "What's the date today?". We expect the tool agent to execute the Python code to get the date.

In [6]:
from agentscope.message import Msg

# The first question
q1_msg = Msg("user", "What's the date today?", role="user")
q1_res = agent(q1_msg)

########################### RAW RESPONSE FROM MODEL ############################
{
    "thought": "I can use Python's datetime module to get the current date.",
    "speak": "Let me check that for you.",
    "function": [
        {
            "name": "execute_python_code",
            "arguments": {
                "code": "from datetime import date\ntoday = date.today()\nprint(today)"
            }
        }
    ]
}
############################### EXECUTE FUNCTION ###############################
FUNCTION NAME: execute_python_code
FUNCTION ARGS: 
{
    "code": "from datetime import date\ntoday = date.today()\nprint(today)"
}
EXECUTE RESULT: 
	STATUS: ('SUCCESS',)
	CONTENT: 2024-03-25

#################################END EXECUTION##################################
Assistant: Let me check that for you.
Assistant: Today's date is March 25, 2024.


The second question is "What's the weather like today?". We expect the tool agent to use the web search API to get the weather information.

In [7]:
q2_msg = Msg("user", "What's the weather like today in Beijing?", role="user")
q2_res = agent(q2_msg)

########################### RAW RESPONSE FROM MODEL ############################
{
    "thought": "I need to search the internet to find the current weather in Beijing.",
    "speak": "Let me check the weather for you.",
    "function": [
        {
            "name": "bing_search",
            "arguments": {
                "question": "current weather in Beijing"
            }
        }
    ]
}
############################### EXECUTE FUNCTION ###############################
FUNCTION NAME: bing_search
FUNCTION ARGS: 
{
    "question": "current weather in Beijing"
}
Assistant: Let me check the weather for you.
EXECUTE RESULT: 
	STATUS: ('SUCCESS',)
	CONTENT: [{'title': 'Beijing, Beijing Municipality, China 14 day weather forecast', 'link': 'https://www.timeanddate.com/weather/china/beijing/ext', 'snippet': 'Currently: 69 °F. Haze. (Weather station: Beijing, China). See more current weather. Beijing Extended Forecast with high and low temperatures. °F. Mar 18 – Mar 24. Lo:41. Sat, 23. H

The above provide two example questions, and you can try to interact with the tool agent by building a conversation with it as follows. 
```python
from agentscope.agents import UserAgent

user = UserAgent(name="User")
x = None
while True:
    x = user(x)
    if x.content == "exit":
        break
    x = agent(x)
```

We provide complete code for building a conversation with the tool agent in [code/main.py](./code/main.py). Feel free to try it and implement your own tool agent!